In [1]:
import numpy as np
import pandas as pd

# Weather problem

## Data

This is a toy dataset of weather data and its relationship with playing golf.

In [2]:
data = pd.DataFrame(
{"Outlook":     ["R", "R", "O", "S", "S", "S", "O", "R", "S", "S"],
 "Temperature": ["H", "H", "H", "M", "C", "C", "C", "M", "M", "M"],
 "Humidity":    ["H", "H", "H", "H", "N", "N", "N", "H", "N", "N"],
 "Windy":       ["F", "T", "F", "F", "F", "T", "T", "F", "T", "F"],
 "Play_Golf":   ["N", "N", "Y", "Y", "Y", "N", "Y", "N", "N", "Y"]
}
)

# Mapping the letters in the dataframe to numbers
data["Outlook"] = data["Outlook"].map({"R":0, "O": 1, "S": 2})
data["Temperature"] = data["Temperature"].map({"C":0, "M": 1, "H": 2})
data["Humidity"] = data["Humidity"].map({"N":0, "H": 1})
data["Windy"] = data["Windy"].map({"F":0, "T": 1})
data["Play_Golf"] = data["Play_Golf"].map({"N":0, "Y": 1})

data.head(n=10)

,Humidity,Outlook,Play_Golf,Temperature,Windy
0,1,0,0,2,0
1,1,0,0,2,1
2,1,1,1,2,0
3,1,2,1,1,0
4,0,2,1,0,0
5,0,2,0,0,1
6,0,1,1,0,1
7,1,0,0,1,0
8,0,2,0,1,1
9,0,2,1,1,0


## Computing information gain for weather data

Here we would manually compute information gain for different features. We will also validate that our computation is correct by computing information gain for two edge cases (i.e. a pure split, random split). We will see that a pure split gives an `information gain=1` and random split gives `information gain=0`.

In [3]:

def compute_infogain(data, feature, label_col_name):
    """
    This function computes the information gain
    IG(Y|X) = H(Y) - H(Y|X)
    data: A pd.DataFrame container all data
    grouped_data: A groupby object that groups Play_GOLF value counts by some feature
    """

    grouped_data = data.groupby(feature)[label_col_name].value_counts()
    
    h_y = compute_entropy(data[label_col_name])
    h_y_given_x = 0
    #print(grouped_data)
    for k in grouped_data.keys():
        # k is a tuple, which has the feature index followed by label index (groupby object)
        k_f, k_y = k
        h_y_given_x += (grouped_data[k_f][k_y].sum()*1.0/data.shape[0])* compute_entropy_with_counts(grouped_data[k_f])
    
    return h_y - h_y_given_x
    
def compute_entropy(ser):
    """
    This function computes the entropy
    """
    total = 0
    counts = ser.value_counts()
    for k in ser.unique():
        #print(k)
        total += -(counts[k]*1.0/ser.shape[0])*np.log2(counts[k]*1.0/ser.shape[0])
        
    return total

def compute_entropy_with_counts(data_counts):
    """
    This function computes the entropy
    """
    total = 0
    for k in data_counts:
        total += -(k*1.0/data_counts.sum())*np.log2(k*1.0/data_counts.sum())
       
    return total

## Making sure computations are correct

## The answer should be 1.0 because the data will be perfectly split to 0s and 1s
## by splitting according to x
print('Checking information gain with toy data')
toy_data = pd.DataFrame({'x':[0,0,0,0,1,1,1,1], 'y':[0,0,0,0,1,1,1,1]})
print('\tInformation Gain for Toy data is: Actual ({}) Expected ({})'.format(
    compute_infogain(toy_data, 'x', "y"), 1.0
))

## The answer should be 0.0 because the data will be random after splitting according to x
toy_data = pd.DataFrame({'x':[1,0,1,0,1,0,1,0], 'y':[0,0,0,0,1,1,1,1]})
print('\tInformation Gain for Toy data is: Actual ({}) Expected ({})\n'.format(
    compute_infogain(toy_data, 'x', "y"), 0.0
))

print('Checking information gain with actual data')
print('\tEntropy for Play_Golf is: {}\n'.format(compute_entropy(data["Play_Golf"])))

for col in ["Outlook", "Temperature", "Humidity", "Windy"]:
    print('\tInformation gain for feature {} is : {}'.format(col, compute_infogain(data, col, "Play_Golf")))

Checking information gain with toy data
	Information Gain for Toy data is: Actual (1.0) Expected (1.0)
	Information Gain for Toy data is: Actual (0.0) Expected (0.0)

Checking information gain with actual data
	Entropy for Play_Golf is: 1.0

	Information gain for feature Outlook is : 0.5145247027726657
	Information gain for feature Temperature is : 0.0490224995673062
	Information gain for feature Humidity is : 0.02904940554533142
	Information gain for feature Windy is : 0.12451124978365313


## Computing the tree

Let us now create a decision tree, which uses information gain to find the best split. The algorithm is as follows:

1. Find a feature to split data (based on the information gain)
2. Split/Partition data into `n` sets depending on the unique values
3. For each partition
  * If termination condition is not met
    * Repeat step 1
    * Repeat step 2
    
    
Following code is adopted from [this repository](https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb). This code is a more general version, where the tree can have an arbitrary number of children. 

**Note** : This tree model cannot work with continuous features, but only discrete/categorical features.

### Tree related objects

In [4]:
class DecisionNode:

    def __init__(self, feature, value, data=None, children=None):
        self.feature = feature
        self.value = value
        self.data = data
        self.children = children

    def set_parent(self, parent):
        self.parent = parent

    def set_data(self, data):
        self.data = data
        
class LeafNode:
    
    def __init__(self, data, label_column):
        self.prediction = data[label_column].value_counts().idxmax()

### Tree related helper functions

In [5]:
def partition_data(feature, values, data_to_split):
    """
    Splits the data such that each dataframe in the returned list 
    has the same value for the selected feature
    """
    data_splits = {}
    for v in values:
        true_data = data_to_split.loc[data_to_split[feature]==v,:]
        data_splits[v] = true_data
    return data_splits


def find_best_feature(data_to_split, features, label_column):
    """
    This function finds the best feature to split that maximizes
    the information gain for a given dataframe
    """
    #feature_infogain_tuples = []
    max_feature, max_ig = None, -1e10
    for f in features:
        #partitions = partition_data(f, set(data_to_split[f].tolist()), data_to_split)
        ig = compute_infogain(data_to_split, f, label_column)
        if ig >= max_ig:
            max_feature = f
            max_ig = ig
    return max_feature, max_ig
     

### Recursive tree building method

In this `build_tree` function, I use two termination conditions to terminate the growth of a branch
* The number of datapoints in a branch is less than `min_leaf_count`
* The information gained by splitting is less than `ig_tol`
* The depth of a branch is greater than `max_depth`

In [6]:
def build_tree(data_to_split, label_column, features, best_feature=None, value=None, depth=0, min_leaf_count=3, ig_tol=1e-10, max_depth=5):
    """
    This Function computes the sub tree recursively. This is a more general tree model
    where there can be arbitrary number of children for a given node
    :param data_to_split: pd.DataFrame
    :param label_column: str (Column name of Y)
    :param features: list of str (Column names of X)
    :param best_feature: str (previous best feature fed recursively to build the tree)
    :param value: int (previous value of the feature fed recursively)
    :param depth: int (previous depth of the tree fed recursively)
    :param min_leaf_count: int (minimum number of datapoints in a leaf)
    :param ig_tol: float (information gain tolerance to make a split)
    :param max_depth: int (maximum depth allowed in the tree)
    """
    
    # Sanity check (ig_tol needs to be > 0). Otherwise
    # data having the same label before and after the split 
    # (information gain =0) will create many useless branches
    if ig_tol<=0.0:
        print('ig_tol needs to be > 0')
        return
    
    # Termination condition 1 (minimum count on leaf)
    if data_to_split.shape[0]<=min_leaf_count:
        print('Too little data. Terminating growth ...')
        children = [LeafNode(data_to_split, label_column)]
        return DecisionNode(best_feature, value, data_to_split, children=children)
    
    # Finding the next best feature to split the data on
    next_best_feature, infogain = find_best_feature(data_to_split, features, label_column)
    feature_unique_values = list(set(data_to_split[next_best_feature].tolist()))
    
    # Termination condition 2 (minimum information gain)
    if infogain < ig_tol or next_best_feature is None:
        print('Too little information gain. Terminating growth ...')
        children = [LeafNode(data_to_split, label_column)]
        return DecisionNode(best_feature, value, data_to_split, children=children)
    
    next_depth = depth + 1
    
    # Termination condition 3 (depth)
    if depth >= max_depth:
        print('Too deep. Terminating growth ...')
        children = [LeafNode(data_to_split, label_column)]
        return DecisionNode(best_feature, value, data_to_split, children=children)
    
    print('Choosing {} as the best feature with {} information gain at depth {}'.format(next_best_feature, infogain, next_depth))
    # Partition the data according to the selected features values
    parts_dict = partition_data(next_best_feature, feature_unique_values, data_to_split)
    
    # For each partition create a child, where child recursively calls build_tree
    children = []
    
    for attr, p in parts_dict.items():
        print('\tCreating child node {}={} having {} data points...'.format(next_best_feature, attr, p.shape[0]))
        children.append(build_tree(p, label_column, features, next_best_feature, attr, next_depth, min_leaf_count, ig_tol, max_depth))
        
    # Return the node
    return DecisionNode(best_feature, value, data_to_split, children=children)

## Running, building the tree

In [7]:
# Returns the root node
my_tree = build_tree(data, "Play_Golf", ["Outlook", "Temperature", "Humidity", "Windy"], ig_tol=1e-3)

Choosing Outlook as the best feature with 0.5145247027726657 information gain at depth 1
	Creating child node Outlook=0 having 3 data points...
Too little data. Terminating growth ...
	Creating child node Outlook=1 having 2 data points...
Too little data. Terminating growth ...
	Creating child node Outlook=2 having 5 data points...
Choosing Windy as the best feature with 0.9709505944546686 information gain at depth 2
	Creating child node Windy=0 having 3 data points...
Too little data. Terminating growth ...
	Creating child node Windy=1 having 2 data points...
Too little data. Terminating growth ...


## Printing the tree
The tree is printed such that the lines with the same indentation are at the same depth in the tree. For example the tree,

`-------------N1
       0--> /  \ <--1
           N2  N3
     0--> / \ <--1
         N4 N5            `
         
will be printed as,

`Root
 N1 = 0
  N2 = 0
   N4
  N2 = 1
   N5
 N1 = 1
  N3     `

In [8]:
print('Printing the decision tree ...')
def print_tree(node, spacing=''):
    """
    This function recursively prints the tree with indentation
    """
    spacing += ' '
    
    if isinstance(node, LeafNode):
        print(spacing, 'Leaf Prediction: ', node.prediction)
        return 
    
    if node.feature==None and node.value==None:
        print(spacing, "Root")
    else:
        print(spacing, "Node",node.feature, '=', node.value)
        
    for c in node.children:
        print_tree(c, spacing)


print_tree(my_tree)

Printing the decision tree ...
  Root
   Node Outlook = 0
    Leaf Prediction:  0
   Node Outlook = 1
    Leaf Prediction:  1
   Node Outlook = 2
    Node Windy = 0
     Leaf Prediction:  1
    Node Windy = 1
     Leaf Prediction:  0


## Predicting with the tree

In [9]:
def predict_datapoint(test_point, my_tree):
    """
    This function computes the prediction for a given data point
    """
    
    for c in my_tree.children:
        # If we came to a leaf node, return the prediction given by the leaf node
        if isinstance(c, LeafNode):
            return c.prediction
        
        # If the feature value matches the value of the data point, keep diving
        if test_point[c.feature] == c.value:
            return predict_datapoint(test_point, c)
    

In [10]:
datapoint = {"Outlook": 0, "Humidity": 0, "Temperature": 2, "Windy": 0}
print('Predicted label for {}'.format(datapoint))

pred = predict_datapoint(datapoint, my_tree)
print('\tPredicted label (Play_Golf): {}'.format(False if pred==0 else True))

Predicted label for {'Temperature': 2, 'Windy': 0, 'Outlook': 0, 'Humidity': 0}
	Predicted label (Play_Golf): False


# Finding the disguised king

## Data

In [11]:
king_data = pd.DataFrame(
{"Castle":     ["Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N"],
 "Slow":       ["Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N"],
 "Gold_Tooth": ["Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y"],
 "Greedy":  ["N", "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y"],
 "Is_King":    ["Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N"]
}
)

king_data["Castle"] = king_data["Castle"].map({"N":0, "Y": 1})
king_data["Slow"] = king_data["Slow"].map({"N":0, "Y": 1})
king_data["Greedy"] = king_data["Greedy"].map({"N":0, "Y": 1})
king_data["Gold_Tooth"] = king_data["Gold_Tooth"].map({"N":0, "Y": 1})
king_data["Is_King"] = king_data["Is_King"].map({"N":0, "Y": 1})

king_data.head(n=10)


,Castle,Gold_Tooth,Greedy,Is_King,Slow
0,1,1,0,1,1
1,0,1,1,1,0
2,0,1,0,1,1
3,1,1,1,1,0
4,1,1,0,1,1
5,0,1,1,0,0
6,0,1,0,0,1
7,1,0,1,0,0
8,0,1,0,0,1
9,0,1,1,0,0


## Building the tree

In [12]:
print('='*10, 'Building tree', '='*10, '\n')
king_tree = build_tree(king_data, "Is_King", ["Castle", "Slow", "Greedy", "Gold_Tooth"], ig_tol=1e-3)
print('\n','='*10, 'Printing tree', '='*10)
print_tree(king_tree)

========== Building tree ========== 

Choosing Castle as the best feature with 0.12451124978365313 information gain at depth 1
	Creating child node Castle=0 having 6 data points...
Too little information gain. Terminating growth ...
	Creating child node Castle=1 having 4 data points...
Choosing Gold_Tooth as the best feature with 0.8112781244591328 information gain at depth 2
	Creating child node Gold_Tooth=0 having 1 data points...
Too little data. Terminating growth ...
	Creating child node Gold_Tooth=1 having 3 data points...
Too little data. Terminating growth ...

 ========== Printing tree ==========
  Root
   Node Castle = 0
    Leaf Prediction:  0
   Node Castle = 1
    Node Gold_Tooth = 0
     Leaf Prediction:  0
    Node Gold_Tooth = 1
     Leaf Prediction:  1


## Overfitting and regularization

Let's see what happens when we increase the amount of data without regularization. You will see how that leads to a massive tree!

In [13]:
aug_king_data = pd.DataFrame(
{"Castle":     [
    "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
    "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
    "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N",
    "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
    "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
],
 "Slow":       [
     "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N",
     "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N",
     "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
     "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N",
     "Y", "N", "N", "Y", "Y", "N", "N", "Y", "N", "N"
 ],
 "Gold_Tooth": [
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y",
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y",
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y",
     "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y",
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y"
 ],
 "Greedy":  [
     "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y",
     "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y",
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y",
     "N", "Y", "N", "Y", "N", "Y", "N", "Y", "N", "Y",
     "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "Y", "Y"
 ],
 "Is_King":    [
     "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N",
     "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N",
     "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N",
     "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N",
     "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N"
 ]
}
)

aug_king_data["Castle"] = aug_king_data["Castle"].map({"N":0, "Y": 1})
aug_king_data["Slow"] = aug_king_data["Slow"].map({"N":0, "Y": 1})
aug_king_data["Greedy"] = aug_king_data["Greedy"].map({"N":0, "Y": 1})
aug_king_data["Gold_Tooth"] = aug_king_data["Gold_Tooth"].map({"N":0, "Y": 1})
aug_king_data["Is_King"] = aug_king_data["Is_King"].map({"N":0, "Y": 1})

print('='*10, 'Building tree', '='*10, '\n')
aug_king_tree = build_tree(aug_king_data, "Is_King", ["Castle", "Slow", "Greedy", "Gold_Tooth"],min_leaf_count=1, ig_tol=1e-20,max_depth=10)
print('\n','='*10, 'Printing tree', '='*10)
print_tree(aug_king_tree)

========== Building tree ========== 

Choosing Castle as the best feature with 0.09845845811405807 information gain at depth 1
	Creating child node Castle=0 having 29 data points...
Choosing Gold_Tooth as the best feature with 0.00480527328186231 information gain at depth 2
	Creating child node Gold_Tooth=0 having 4 data points...
Choosing Slow as the best feature with 0.12255624891826566 information gain at depth 3
	Creating child node Slow=0 having 3 data points...
Too little information gain. Terminating growth ...
	Creating child node Slow=1 having 1 data points...
Too little data. Terminating growth ...
	Creating child node Gold_Tooth=1 having 25 data points...
Choosing Slow as the best feature with 0.005646310646669206 information gain at depth 3
	Creating child node Slow=0 having 18 data points...
Too little information gain. Terminating growth ...
	Creating child node Slow=1 having 7 data points...
Choosing Greedy as the best feature with 0.19811742113040343 information gain at

## Tree model with regularization
We set `min_leaf_count`=5, `ig_tol`=1e-2 and `max_depth`=3 as regularization steps. You can see that the new tree is way simpler and interpretable than the previous tree.

In [14]:
print('='*10, 'Building tree', '='*10, '\n')
aug_king_tree = build_tree(aug_king_data, "Is_King", ["Castle", "Slow", "Greedy", "Gold_Tooth"],min_leaf_count=5, ig_tol=1e-2,max_depth=3)
print('\n','='*10, 'Printing tree', '='*10)
print_tree(aug_king_tree)

========== Building tree ========== 

Choosing Castle as the best feature with 0.09845845811405807 information gain at depth 1
	Creating child node Castle=0 having 29 data points...
Too little information gain. Terminating growth ...
	Creating child node Castle=1 having 21 data points...
Choosing Slow as the best feature with 0.14026267083366217 information gain at depth 2
	Creating child node Slow=0 having 7 data points...
Choosing Gold_Tooth as the best feature with 0.2916919971380597 information gain at depth 3
	Creating child node Gold_Tooth=0 having 2 data points...
Too little data. Terminating growth ...
	Creating child node Gold_Tooth=1 having 5 data points...
Too little data. Terminating growth ...
	Creating child node Slow=1 having 14 data points...
Choosing Gold_Tooth as the best feature with 0.04957603870374783 information gain at depth 3
	Creating child node Gold_Tooth=0 having 3 data points...
Too little data. Terminating growth ...
	Creating child node Gold_Tooth=1 having

## Exercises:
* Change the LeafNode, so that it outputs the prediction probabilities in addition to the class
* John thought of a new feature, which is the `distance traveled in a day`. Rewrite (below) a decision tree that can work with continuous values (you can start with a simple method such as binning) to discrete values. 